# My Friendly Neighborhood Project: Safe School Districts in City of New York

## Introduction
This project attempts to identify safe school disctricts in cities like New York.

## Business Problem
Large cities of the size of New York  attracts people from all over the world. There are people who move in permanently as immigrants,
some move for work, some to reunite with family and for various other reasons. For all these people, especially those with families, 
with kids of school going age, it is vital to identify safe neighourhoods.
This project attempts to solve this problem by identifying high crime rate areas which serves as an indicator for newcomers to watch out for before making decisions regarding renting or buying real estate. It will help them avoid places with relatively higher crime rates.

## Target Audience

This project aims to benefit new comers to New York City. The data visualization will help them take an informed decision about the place to look for apartments and settle down in a safe neighbourhood.

## Data
Listed are the data sources for help with this project :

1. List of boroughs and neighborhoods in the state of NYC. Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.
Here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

2. GeoJSON data for New York : 
https://cocl.us/new_york_dataset

3. API calls to get location data from the following source:
https://Foursquare.com




## Table of Contents

1. Download and Explore Dataset

2. Explore Neighborhoods in New York City

3. Analyze Each Neighborhood

4. Cluster Neighborhoods

To start with, we download the necessary library files, fetch the data and examine the structure.

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Download and Examine the data structure

In [13]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [14]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Transform the data into a pandas dataframe

In [15]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [16]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [17]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Locate the coordinates of New York City

In [19]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [20]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Define Foursquare Credentials

In [21]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))
manhattan_data.loc[0, 'Neighborhood']

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [22]:
CLIENT_ID = '1RL2YNRCFD1QA1UF1RXMIITWHR3BHTOEF2HWXJY5BB3PBB4U' # your Foursquare ID
CLIENT_SECRET = 'Y05EWS0YV4D05CCF24VGPW5LBVULOWIWJYPOMMMCVZASJCDO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [23]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
LIMIT=100     # Number of venues returned bz Foursquare API
radius = 500  # defined radius of 500 m
# Create URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format( CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


'https://api.foursquare.com/v2/venues/explore?client_id=1RL2YNRCFD1QA1UF1RXMIITWHR3BHTOEF2HWXJY5BB3PBB4U&client_secret=Y05EWS0YV4D05CCF24VGPW5LBVULOWIWJYPOMMMCVZASJCDO&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

### Get the venues that are in reasonable walking distance of 500 meters.

In [24]:
results = requests.get(url).json()

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
manhattan_venues.groupby('Neighborhood').count()

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,45,45,45,45,45,45
East Village,100,100,100,100,100,100


In [29]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 330 uniques categories.


### Analyze Each Neighborhood - Identify Categories by Grouping

In [30]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,S

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Italian Restaurant,Food Truck,Wine Shop,Sandwich Place,Shopping Mall,Fountain
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Cosmetics Shop,Gym,Bar,Spa,French Restaurant,Bookstore,Yoga Studio
2,Central Harlem,Chinese Restaurant,African Restaurant,French Restaurant,Gym / Fitness Center,Seafood Restaurant,Cosmetics Shop,American Restaurant,Cafeteria,Market,Beer Bar
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Seafood Restaurant,Theater,Hotel,French Restaurant
4,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Salon / Barbershop,Noodle House,Ice Cream Shop,Hotpot Restaurant,Bubble Tea Shop


## Cluster the Neighborhoods - Use **k-Means** clustering technique

In [33]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 0, 1, 1, 0, 0, 4, 1, 3], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Coffee Shop,Discount Store,Clothing Store,Big Box Store,Supplement Shop,Steakhouse,Spa,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Vietnamese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Salon / Barbershop,Noodle House,Ice Cream Shop,Hotpot Restaurant,Bubble Tea Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,Pizza Place,Grocery Store,Chinese Restaurant,New American Restaurant,Spanish Restaurant,Latin American Restaurant,Supermarket
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Lounge,Pizza Place,Restaurant,Chinese Restaurant,Frozen Yogurt Shop,Pharmacy,Park,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Deli / Bodega,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Caribbean Restaurant,Bakery,Chinese Restaurant,School


## Results - Visualize using the Map

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine The Clusters 

### Cluster 1

In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,Chinese Restaurant,African Restaurant,French Restaurant,Gym / Fitness Center,Seafood Restaurant,Cosmetics Shop,American Restaurant,Cafeteria,Market,Beer Bar
8,Upper East Side,Italian Restaurant,Exhibit,Coffee Shop,Juice Bar,Bakery,Gym / Fitness Center,Cocktail Bar,Hotel,French Restaurant,Boutique
13,Lincoln Square,Theater,Concert Hall,Café,Italian Restaurant,Gym / Fitness Center,Plaza,Opera House,French Restaurant,Performing Arts Venue,Indie Movie Theater
14,Clinton,Theater,Italian Restaurant,American Restaurant,Gym / Fitness Center,Hotel,Gym,Spa,Wine Shop,Indie Theater,Coffee Shop
15,Midtown,Hotel,Clothing Store,Steakhouse,Theater,Cocktail Bar,Spa,American Restaurant,Coffee Shop,Bakery,Food Truck
18,Greenwich Village,Italian Restaurant,French Restaurant,Sushi Restaurant,Clothing Store,Chinese Restaurant,Boutique,Bakery,Café,Seafood Restaurant,Indian Restaurant
21,Tribeca,Italian Restaurant,Café,Park,Spa,American Restaurant,Boutique,Wine Bar,Gym,Coffee Shop,Greek Restaurant
22,Little Italy,Bakery,Café,Sandwich Place,Chinese Restaurant,Clothing Store,Seafood Restaurant,Salon / Barbershop,Bubble Tea Shop,Italian Restaurant,Ice Cream Shop
23,Soho,Clothing Store,Boutique,Women's Store,Men's Store,Shoe Store,Art Gallery,Mediterranean Restaurant,Italian Restaurant,Coffee Shop,Seafood Restaurant
24,West Village,Italian Restaurant,Cosmetics Shop,New American Restaurant,Park,Jazz Club,Gastropub,Wine Bar,French Restaurant,Bakery,American Restaurant


### Cluster 2

In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Salon / Barbershop,Noodle House,Ice Cream Shop,Hotpot Restaurant,Bubble Tea Shop
5,Manhattanville,Mexican Restaurant,Park,Coffee Shop,Italian Restaurant,Seafood Restaurant,Food & Drink Shop,Lounge,Grocery Store,Bike Trail,Sushi Restaurant
9,Yorkville,Italian Restaurant,Gym,Bar,Coffee Shop,Pizza Place,Deli / Bodega,Japanese Restaurant,Diner,Sushi Restaurant,Mexican Restaurant
10,Lenox Hill,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Sporting Goods Shop,Burger Joint,Gym,Deli / Bodega,Salad Place
12,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Indian Restaurant,Wine Bar,Bakery,Cosmetics Shop,Vegetarian / Vegan Restaurant,Yoga Studio,Sushi Restaurant
16,Murray Hill,Coffee Shop,Hotel,French Restaurant,Italian Restaurant,Gym,Bar,Sandwich Place,American Restaurant,Spa,Japanese Restaurant
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Seafood Restaurant,Theater,Hotel,French Restaurant
19,East Village,Bar,Wine Bar,Ice Cream Shop,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Pizza Place,Ramen Restaurant,Speakeasy
25,Manhattan Valley,Indian Restaurant,Coffee Shop,Pizza Place,Yoga Studio,Mexican Restaurant,Café,Bar,Thai Restaurant,Playground,Deli / Bodega
27,Gramercy,Italian Restaurant,Cocktail Bar,Bagel Shop,Thrift / Vintage Store,Pizza Place,Grocery Store,American Restaurant,Wine Shop,Mexican Restaurant,Bar


### Cluster 3

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Park,Playground,Basketball Court,Harbor / Marina,Cocktail Bar,Coffee Shop,Heliport,Pet Service,Farmers Market


### Cluster 4

In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Park,Sandwich Place,Coffee Shop,Outdoors & Recreation,Farmers Market,Supermarket,Metro Station,School,Scenic Lookout,Greek Restaurant
26,Morningside Heights,Coffee Shop,Park,Bookstore,American Restaurant,Food Truck,Tennis Court,Burger Joint,Sandwich Place,Deli / Bodega,College Cafeteria
28,Battery Park City,Park,Coffee Shop,Hotel,Gym,Italian Restaurant,Food Truck,Wine Shop,Sandwich Place,Shopping Mall,Fountain
29,Financial District,Coffee Shop,Steakhouse,Hotel,Bar,Gym,Wine Shop,Pizza Place,Japanese Restaurant,Park,Italian Restaurant


### Cluster 5

In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Discount Store,Clothing Store,Big Box Store,Supplement Shop,Steakhouse,Spa,Shoe Store,Seafood Restaurant,Sandwich Place
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Pizza Place,Grocery Store,Chinese Restaurant,New American Restaurant,Spanish Restaurant,Latin American Restaurant,Supermarket
3,Inwood,Mexican Restaurant,Café,Lounge,Pizza Place,Restaurant,Chinese Restaurant,Frozen Yogurt Shop,Pharmacy,Park,Bakery
4,Hamilton Heights,Deli / Bodega,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Caribbean Restaurant,Bakery,Chinese Restaurant,School
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Café,French Restaurant,Steakhouse,Spanish Restaurant,Taco Place
20,Lower East Side,Café,Chinese Restaurant,Coffee Shop,Art Gallery,Ramen Restaurant,Bakery,Japanese Restaurant,Shoe Store,Cocktail Bar,Pizza Place
36,Tudor City,Park,Mexican Restaurant,Sushi Restaurant,Café,Asian Restaurant,Dog Run,Diner,Deli / Bodega,Hotel,Greek Restaurant


### 5 Clusters depicted.